# Municipality DataFram collation

In [7]:
# Imports
import pandas as pd
import numpy as np
import json
from scipy.optimize import minimize
from geopy.distance import geodesic
import concurrent.futures
from tqdm import tqdm

In [14]:
# Read data
municipalities = pd.read_csv('../data/processed/pops_left.csv', encoding='utf-8')
hospitals = pd.read_csv('../data/processed/hospitals_collated_reduced.csv')

with open('../data/raw/gemeinden_wus.geojson', 'r') as f:
    geojson = json.load(f)

# Determine geographic center of municipalities
We want to assign a hospital to each municipality. We will do so by assigning the geographically closest hospital to each municipality. Therefore we have to calculate the geographic center for each municipality using the data from the geojson file.

In [9]:

# def geodesic_distance_sum(point, coords):
#     """ Calculate the sum of geodesic distances from the point to all coordinates """
#     lat, lon = point
#     return sum(geodesic((lat, lon), coord).meters for coord in coords)

# def geographic_median(coords):
#     """
#     Find the geographic median (point equidistant to all given coordinates)
#     using geodesic distance.
#     """
#     # Start with the arithmetic mean as an initial guess
#     initial_guess = np.mean(coords, axis=0)
    
#     # Minimize the sum of geodesic distances to all points
#     result = minimize(geodesic_distance_sum, initial_guess, args=(coords,), method='Nelder-Mead')
    
#     return result.x  # Return the optimized latitude and longitude

# geocenter = dict()

# for mun in geojson['features']:
#     name = mun['properties']['gemeinde_NAME']
#     geo_coords = mun['geometry']['coordinates']
#     coords = []

#     c_list = [c for c in geo_coords[0][0]]
#     [coords.append((latitute, longitude)) for longitude, latitute, _ in c_list]
    
#     center = geographic_median(coords)
#     geocenter[name] = center

# #print(geojson['features'][0]['properties']['gemeinde_NAME'])
# # print(geojson['features'][0]['geometry']['coordinates'])


In [10]:
# def geographic_median(coords):
#     """
#     Find the geographic median (point equidistant to all given coordinates)
#     using geodesic distance.
#     """
#     # Start with the arithmetic mean as an initial guess
#     initial_guess = np.mean(coords, axis=0)
    
#     # Minimize the sum of geodesic distances to all points
#     result = minimize(geodesic_distance_sum, initial_guess, args=(coords,), method='Nelder-Mead')
    
#     return result.x  # Return the optimized latitude and longitude

# def process_municipality(mun):
#     name = mun['properties']['gemeinde_NAME']
#     geo_coords = mun['geometry']['coordinates']
#     coords = []

#     c_list = [c for c in geo_coords[0][0]]
#     [coords.append((latitute, longitude)) for longitude, latitute, _ in c_list]

#     center = geographic_median(coords)
#     return name, center

# geocenter = {}

# num_municipalities = len(geojson['features'])
# pbar = tqdm(total=num_municipalities, desc="Processing municipalities")

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     futures = {executor.submit(process_municipality, mun): mun for mun in geojson['features']}
#     for future in concurrent.futures.as_completed(futures):
#         mun = futures[future]
#         try:
#             name, center = future.result()
#             geocenter[name] = center
#             pbar.update(1)
#         except Exception as e:
#             print(f"Error processing municipality {mun['properties']['gemeinde_NAME']}: {e}")
#             pbar.update(1)

# pbar.close()

try without optimisation an see whether using the mean is good enogugh. This will run much faster.

In [11]:
def geographic_median(coords):
    """
    Find the geographic median (point equidistant to all given coordinates)
    using geodesic distance.
    """
    initial_guess = np.mean(coords, axis=0)
    return initial_guess

geocenter = dict()

for mun in geojson['features']:
    name = mun['properties']['gemeinde_NAME']
    geo_coords = mun['geometry']['coordinates']
    coords = []

    c_list = [c for c in geo_coords[0][0]]
    [coords.append((latitute, longitude)) for longitude, latitute, _ in c_list]
    
    center = geographic_median(coords)
    geocenter[name] = center

In [17]:
municipalities['coords'] = municipalities['gemeinde_NAME'].map(geocenter)
municipalities.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
municipalities.head()

,B19BTOT,gemeinde_NAME,coords
0,1981,Aeugst am Albis,"[47.27390000382371, 8.487954663272388]"
1,5721,Obfelden,"[47.26417791798067, 8.41875479814007]"
2,2293,Stadel,"[47.53913755670497, 8.466035447304153]"
3,789,Doppleschwand,"[47.01161188422407, 8.047201928359051]"
4,3280,Entlebuch,"[46.97605417000893, 8.110869044268913]"


In [18]:
municipalities.to_csv('../data/processed/pops_left_coords.csv', index=False)